### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#1. Estudiar similaridad del coseno

In [ ]:
def eval_similaridad(idx):
  j = 0
  print(f'Para el documento N°: {idx}\nDocumento:\n')
  print(newsgroups_train.data[idx])
  # midamos la similaridad coseno con todos los documentos de train
  cossim = cosine_similarity(X_train[idx], X_train)[0]
  # visualicemos cuales son los valores mas parecidos
  print(f'\n\n--------------------------------------------')
  print(f'Los documentos mas cercanos tienen una similaridad de:')
  print(np.sort(cossim)[::-1][1:6])
  mostsim = np.argsort(cossim)[::-1][1:6]
  # el documento original
  orig_class = newsgroups_train.target_names[y_train[idx]]
  print(f'\nEl documento evaluado pertenece a la clase: {orig_class}\n')
  # y los 5 más similares son de las clases:
  for i in mostsim:
    sim_class = newsgroups_train.target_names[y_train[i]]
    print(f'Los 5 documentos mas similares pertenecen a la clases: {sim_class}')
    if sim_class == orig_class:
      j+=1
  print(f'\nDe los 5 documentos, {j} son de la misma clase que el original')

In [ ]:
eval_similaridad(852)

Para el documento N°: 852
Documento:

:P
:P>My favorite reply to the "you are being too literal-minded" complaint is
:P>that if the bible is really inspired by God and if it is really THAT
:P>important to him, then he would make damn certain all the translators and
:P>scribes and people interpreting and copying it were getting it right,
:P>literally.  If not, then why should I put ANY merit at all in something
:P>that has been corrupted over and over and over by man even if it was
:P>originally inspired by God?
:P
:PThe "corrupted over and over" theory is pretty weak.  Comparison of the
:Pcurrent hebrew text with old versions and translations shows that the text
:Phas in fact changed very little over a space of some two millennia.  This
:Pshouldn't be all that suprising; people who believe in a text in this manner
:Pare likely to makes some pains to make good copies.


--------------------------------------------
Los documentos mas cercanos tienen una similaridad de:
[0.47556128 0.2919

In [ ]:
eval_similaridad(1583)

Para el documento N°: 1583
Documento:

Found it! Thanks. I got several offers for help. I appreciate it and
will be contacting those people via e-mail.

Thanks again...


--------------------------------------------
Los documentos mas cercanos tienen una similaridad de:
[0.2859492  0.23523338 0.21219821 0.19131326 0.18752988]

El documento evaluado pertenece a la clase: sci.space

Los 5 documentos mas similares pertenecen a la clases: comp.graphics
Los 5 documentos mas similares pertenecen a la clases: comp.windows.x
Los 5 documentos mas similares pertenecen a la clases: rec.sport.baseball
Los 5 documentos mas similares pertenecen a la clases: misc.forsale
Los 5 documentos mas similares pertenecen a la clases: sci.space

De los 5 documentos, 1 son de la misma clase que el original


In [ ]:
eval_similaridad(343)

Para el documento N°: 343
Documento:


In fact, you probably want to avoid US Government anything for such a
project.  The pricetag is invariably too high, either in money or in
hassles.

The important thing to realize here is that the big cost of getting to
the Moon is getting into low Earth orbit.  Everything else is practically
down in the noise.  The only part of getting to the Moon that poses any
new problems, beyond what you face in low orbit, is the last 10km --
the actual landing -- and that is not immensely difficult.  Of course,
you *can* spend sagadollars (saga- is the metric prefix for beelyuns
and beelyuns) on things other than the launches, but you don't have to.

The major component of any realistic plan to go to the Moon cheaply (for
more than a brief visit, at least) is low-cost transport to Earth orbit.
For what it costs to launch one Shuttle or two Titan IVs, you can develop
a new launch system that will be considerably cheaper.  (Delta Clipper
might be a bit more ex

In [ ]:
eval_similaridad(2000)

Para el documento N°: 2000
Documento:



_Cycle World_ puts one out, but I'm sure it's not very objective.  Try talking
with dealers and the people that hang out there, as well as us.  We love to
give advice.


Most of the bigger banks have a blue book which includes motos -- ask for the
one with RVs in it.


Couldn't help you here.


You're reading it.

----------------------------------------------------------------------------


--------------------------------------------
Los documentos mas cercanos tienen una similaridad de:
[0.22168372 0.22043832 0.20057127 0.19569925 0.19561147]

El documento evaluado pertenece a la clase: rec.motorcycles

Los 5 documentos mas similares pertenecen a la clases: talk.politics.misc
Los 5 documentos mas similares pertenecen a la clases: talk.politics.mideast
Los 5 documentos mas similares pertenecen a la clases: talk.politics.misc
Los 5 documentos mas similares pertenecen a la clases: talk.politics.misc
Los 5 documentos mas similares pertenecen a la

In [ ]:
eval_similaridad(10000)

Para el documento N°: 10000
Documento:

I have to disagree with this.  I have a 92 Z28 with a 350 and a 4-speed auto
w/ overdrive, and it is really better that way.  Chevy autos are reknowned
for their long life and ability to handle copious amount of power.  I live 
in the Dallas area, and a manual would be much harder to drive in the traffic 
here.  Now if I still lived out in the sticks like I used to, a manual would be
more fun.  

Safety-wise, an auto is less distracting...I would hate to have to be    
shifting gears while I was trying to ease into traffic in the freeways here.
Performance-wise, I can hold my own against any stock 5.0 Mustang or 5.0
Camaro w/ a five speed.  

All of this IMHO... :)






--------------------------------------------
Los documentos mas cercanos tienen una similaridad de:
[0.2612887  0.2553084  0.23345549 0.21737891 0.21498468]

El documento evaluado pertenece a la clase: rec.autos

Los 5 documentos mas similares pertenecen a la clases: rec.autos
Lo

In [ ]:
eval_similaridad(674)

Para el documento N°: 674
Documento:

From: Center for Policy Research <cpr>
Subject: Assistance to Palest.people


U.N. General Assembly Resolution 46/201 of 20 December 1991

ASSISTANCE TO THE PALESTINIAN PEOPLE
---------------------------------------------
The General Assembly

Recalling its resolution 45/183 of 21 December 1990

Taking into account the intifadah of the Palestinian people in the
occupied Palestinian territory against the Israeli occupation,
including Israeli economic and social policies and practices,

Rejecting Israeli restrictions on external economic and social
assistance to the Palestinian people in the occupied Palestinian
territory,

Concerned about the economic losses of the Palestinian people as a
result of the Gulf crisis,

Aware of the increasing need to provide economic and social
assistance to the Palestinian people,

Affirming that the Palestinian people cannot develop their
national economy as long as the Israeli occupation persists,

1. Takes note of 

##Analisis de resultados


```
Según los resultados obtenidos, podemos resumir lo siguiente:

doc evaluado     doc de la misma clase (extraidos del top 5 mas similes)
    852                  2
    1583                 1
    343                  4
    2000                 0
    10000                3
    674                  5


Según esto podemos ver que ciertamente la similaridad del coseno logra ubicar documentos similes y
que tiene cierta acertividad, evidentemente dependerá de cuan nutrido es el documento en palabras
ya que se basa en esto directamente, por lo que podemos decir que dependerá mucho de el documento
y el corpus como tal para poder tener buenos resultados.

Sin embargo, tomando en cuenta que es un análisis no tan demandante en cuanto a computo, no tan
"inteligente" y que se usa solo una propiedad de geometria analitica, resulta interesante que
tenga buenos resultados.
```



#2. Entrenar modelos y realizar tunnig de NB y ComplementNB


In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

##Modelo MultinomialNB base

In [8]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

##Modelo MultinomialNB optimizado

In [9]:
tfidfvect = TfidfVectorizer(lowercase=True, norm= "l2", sublinear_tf = False, max_df=0.1, min_df=1)
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [10]:
clf = MultinomialNB(alpha=0.01)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

In [11]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [12]:
f1_score(y_test, y_pred, average='macro')

0.685572214627961

##Modelo ComplementNB base

In [15]:
tfidfvect_c = TfidfVectorizer()
X_train_c = tfidfvect_c.fit_transform(newsgroups_train.data)
y_train_c = newsgroups_train.target
clf_comp = ComplementNB()
clf_comp.fit(X_train_c, y_train_c)
X_test_c = tfidfvect_c.transform(newsgroups_test.data)
y_test_c = newsgroups_test.target
y_pred_c =  clf_comp.predict(X_test_c)
f1_score(y_test_c, y_pred_c, average='macro')

0.692953349950875

##Modelo ComplementNB optimizado

In [45]:
tfidfvect = TfidfVectorizer(lowercase=True, norm= "l2", sublinear_tf = False, max_df=0.15, min_df=1)
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [46]:
clf_comp = ComplementNB(alpha=0.305)
clf_comp.fit(X_train, y_train)

ComplementNB(alpha=0.305)

In [47]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf_comp.predict(X_test)

In [48]:
f1_score(y_test, y_pred, average='macro')

0.7002244870750653

##Analisis de resultados

```
Según los resultados obtenidos, podemos observar que el modelo MultinomialNB pudo ser optimizado su
F1 Score desde 0,58 a 0,68 lo cual es una mejoría notoria, en donde los parámetros más
incidentes fueron max_df (en el vectorizador TfidfVectorizer) y alpha del modelo MultinomialNB.

Por otra parte el modelo ComplementNB, hace un análisis tomando en cuenta las estadísticas del
complemento de cada clase para el cálculo de los pesos del modelo, lo que hace que sea mas estable
sobretodo en datasets desbalanceados, déspues de varias pruebas solo se pudo mejorar su resultado
F1 Score desde 0.6929 a 0.7002
```

#3. Evaluar matriz termino-documento

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
tfidfvect = TfidfVectorizer(max_df=0.1)

In [ ]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [ ]:
X_transpuesta = X_train.T

idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
def eval_simil_td(idx):

  print(f'El termino a evaluar es: {idx2word[idx]}\n')
  # midamos la similaridad coseno con todos los documentos de train
  cossim = cosine_similarity(X_transpuesta[idx], X_transpuesta)[0]
  # visualicemos cuales son los valores mas parecidos
  print(f'Los 5 terminos mas cercanos tienen una similaridad de:')
  print(np.sort(cossim)[::-1][1:6])
  mostsim = np.argsort(cossim)[::-1][1:6]
  # y los 5 más similares:
  print(f'\nLos 5 terminos mas cercanos segun la similaridad de coseno son:\n')
  for i in mostsim:
    sim_class = idx2word[i]
    print(f'{sim_class}')

In [ ]:
eval_simil_td(35179)

El termino a evaluar es: drugs

Los 5 terminos mas cercanos tienen una similaridad de:
[0.36328859 0.36328859 0.31883096 0.28219986 0.28219986]

Los 5 terminos mas cercanos segun la similaridad de coseno son:

amphetamines
chg
drug
hazzards
incarcerating


In [ ]:
eval_simil_td(56630)

El termino a evaluar es: life

Los 5 terminos mas cercanos tienen una similaridad de:
[0.15243419 0.14483868 0.1403533  0.13973247 0.13949106]

Los 5 terminos mas cercanos segun la similaridad de coseno son:

real
god
sex
eternal
hermite


In [ ]:
eval_simil_td(97140)

El termino a evaluar es: work

Los 5 terminos mas cercanos tienen una similaridad de:
[0.14427037 0.14088067 0.13577355 0.13117565 0.11916676]

Los 5 terminos mas cercanos segun la similaridad de coseno son:

fine
spoofing
needed
retraining
installed


In [ ]:
eval_simil_td(50776)

El termino a evaluar es: investment

Los 5 terminos mas cercanos tienen una similaridad de:
[0.51425397 0.46401318 0.44305247 0.36274371 0.34813519]

Los 5 terminos mas cercanos segun la similaridad de coseno son:

benfits
taxpayer
settling
resounding
proxy


In [ ]:
eval_simil_td(36942)

El termino a evaluar es: emergency

Los 5 terminos mas cercanos tienen una similaridad de:
[0.39004987 0.3336042  0.32132347 0.3144143  0.3048889 ]

Los 5 terminos mas cercanos segun la similaridad de coseno son:

receptionist
clinic
tac
vor
hospitalization


##Analisis de resultados

```
Según los resultados obtenidos, podemos observar que en este caso evaluando la matriz
término-documento, la similaridad del coseno sigue ubicando cierta relación entre
términos que tienen un documentos similares, una vez más, es interesante que pueda
conseguir ciertas similitudes aunque sin ser prometedoras, pueden tomarse en cuenta
quizás como un dato base para después de esto ir usando métodos más elaborados.
```